In [2]:
import pandas as pd
import numpy as np
import glob, os, gc
from scipy.stats import skew, kurtosis, linregress
from scipy.optimize import curve_fit
from scipy.fft import rfft
from numpy.polynomial.polynomial import polyfit
import warnings
warnings.filterwarnings("ignore")

# ========== 1. TỰ ĐỘNG TÌM DATA ==========

print("📂 Đang tìm dữ liệu MALLORN...")
input_dirs = glob.glob("/kaggle/input/*")
BASE_PATH = ""
for path in input_dirs:
    if "mallorn" in path.lower() and os.path.isdir(path):
        BASE_PATH = path
        break
if BASE_PATH == "" and len(input_dirs) > 0:
    BASE_PATH = input_dirs[0]
print("BASE_PATH:", BASE_PATH)

# ========== 2. HỖ TRỢ: TDE FIT + BIẾN ĐỔI FOURIER ==========

def fit_tde_shape(t, f, f_peak, t_peak):
    mask = (t > t_peak) & (f > 0.05 * f_peak)
    if np.sum(mask) < 3:
        return -1.0
    t_dec = t[mask]
    f_dec = f[mask]

    def func(ti, t0):
        dt = ti - t0
        return f_peak * np.power(np.maximum(dt / (t_peak - t0), 1e-6), -5/3)

    try:
        popt, _ = curve_fit(
            func, t_dec, f_dec,
            p0=[t_peak - 20],
            bounds=([t_peak - 500], [t_peak - 0.5]),
            maxfev=200,
        )
        resid = f_dec - func(t_dec, *popt)
        return np.sum(resid**2) / len(t_dec)
    except Exception:
        return -1.0

def get_fft_features(flux):
    if len(flux) < 5:
        return [0.0]*4
    f_fft = np.abs(rfft(flux))
    if len(f_fft) < 3:
        return [0.0]*4
    vals = list(f_fft[1:5]) if len(f_fft) >= 5 else list(f_fft[1:])
    if len(vals) < 4:
        vals += [0.0]*(4-len(vals))
    return vals

# ========== 3. HÀM TRÍCH XUẤT FEATURE ==========

def extract_band_features(time, flux, err, filt, z):
    f = {}
    n = len(flux)
    if n < 3:
        return f, None, None

    # Basic stats
    f_max = np.max(flux)
    f_min = np.min(flux)
    f_mean = np.mean(flux)
    f_std = np.std(flux)
    f[f"{filt}_mean"] = f_mean
    f[f"{filt}_std"] = f_std
    f[f"{filt}_max"] = f_max
    f[f"{filt}_min"] = f_min
    f[f"{filt}_skew"] = skew(flux)
    f[f"{filt}_kurt"] = kurtosis(flux)
    f[f"{filt}_mad"] = np.median(np.abs(flux - np.median(flux)))
    f[f"{filt}_amp"] = (f_max - f_min) / (f_mean + 1e-9)
    f[f"{filt}_cv"] = f_std / (f_mean + 1e-9)

    # Các Phân Vị
    pcts = np.percentile(flux, [0,5,10,20,30,40,50,60,70,80,90,95,100])
    for i, p in zip([0,5,10,20,30,40,50,60,70,80,90,95,100], pcts):
        f[f"{filt}_p{i:02d}"] = p
    f[f"{filt}_ratio_p95_p05"] = pcts[11] / (pcts[1] + 1e-9)
    f[f"{filt}_ratio_p95_median"] = pcts[11] / (pcts[6] + 1e-9)

    # Sai Phân / Von Neumann
    diff = np.diff(flux)
    f[f"{filt}_diff_mean"] = np.mean(diff)
    f[f"{filt}_diff_std"] = np.std(diff)
    f[f"{filt}_diff_max"] = np.max(diff)
    f[f"{filt}_diff_min"] = np.min(diff)
    f[f"{filt}_von_neumann"] = np.sum(diff**2) / (np.sum((flux - f_mean)**2) + 1e-9)

    # Tích Lũy
    cum_flux = np.cumsum(np.abs(flux))
    f[f"{filt}_energy"] = cum_flux[-1]
    try:
        slope_cum = linregress(np.arange(n), cum_flux)[0]
        f[f"{filt}_cum_slope"] = slope_cum
    except Exception:
        f[f"{filt}_cum_slope"] = 0.0

    # Time shape (rest-frame)
    t_peak = time[np.argmax(flux)]
    t_min = time.min()
    t_max = time.max()
    f[f"{filt}_time_to_peak"] = t_peak - t_min
    f[f"{filt}_time_from_peak"] = t_max - t_peak

    z_eff = max(float(z), 0.0)
    t_rest = (time - t_peak) / (1.0 + z_eff)
    mask_rise = t_rest < 0
    mask_fall = t_rest > 0

    if np.sum(mask_rise) > 1:
        f[f"{filt}_slope_rise_rest"] = linregress(t_rest[mask_rise], flux[mask_rise])[0]
    else:
        f[f"{filt}_slope_rise_rest"] = 0.0
    if np.sum(mask_fall) > 1:
        f[f"{filt}_slope_fall_rest"] = linregress(t_rest[mask_fall], flux[mask_fall])[0]
    else:
        f[f"{filt}_slope_fall_rest"] = 0.0

    thr = 0.25 * f_max
    above = time[flux >= thr]
    if len(above) >= 2:
        f[f"{filt}_dur_025"] = (above.max() - above.min()) / (1.0 + z_eff)
    else:
        f[f"{filt}_dur_025"] = 0.0

    # Log-decay slope
    log_flux = np.log10(np.maximum(flux, 1e-6))
    if np.sum(mask_fall) > 2:
        f[f"{filt}_logdecay_slope"] = linregress(time[mask_fall], log_flux[mask_fall])[0]
    else:
        f[f"{filt}_logdecay_slope"] = 0.0

    # Poly2
    t_norm = (time - time.mean()) / (1.0 + z_eff)
    if len(t_norm) >= 6:
        c0, c1, c2 = polyfit(t_norm, flux, 2)
        f[f"{filt}_poly2_c0"] = c0
        f[f"{filt}_poly2_c1"] = c1
        f[f"{filt}_poly2_c2"] = c2

    # BIẾN ĐỔI FOURIER
    fft_vals = get_fft_features(flux)
    for i, val in enumerate(fft_vals):
        f[f"{filt}_fft_{i}"] = val

    # TỈ SỐ TÍN HIỆU TRÊN NHIỄU
    snr = flux / (err + 1e-9)
    f[f"{filt}_snr_mean"] = np.mean(snr)
    f[f"{filt}_snr_max"] = np.max(snr)
    f[f"{filt}_snr_p90"] = np.percentile(snr, 90)
    f[f"{filt}_n_snr_gt5"] = np.sum(snr > 5)

    # TDE power-law
    if filt in ["g", "r", "u"]:
        f[f"{filt}_tde_chisq"] = fit_tde_shape(time, flux, f_max, t_peak)

    return f, t_peak, f_max

# ========== 4. QUY TRÌNH XỬ LÝ TỪNG VẬT THỂ ==========

def process_pipeline(df_lc, df_log):
    print(" -> Trích xuất features...")
    EXTINCTION = {"u": 4.81, "g": 3.64, "r": 2.70, "i": 2.06, "z": 1.58, "y": 1.31}
    df = df_lc.gộp(df_log[["object_id", "EBV", "Z"]], on="object_id", how="left").fillna(0)
    df["R"] = df["Filter"].map(EXTINCTION)
    df["Flux_Corr"] = df["Flux"] * (10 ** (0.4 * df["R"] * df["EBV"].clip(0, 2.0)))

    features = []
    for obj_id, grp in df.groupby("object_id"):
        row = {"object_id": obj_id}
        grp = grp.sort_values("Time (MJD)")
        z = float(grp["Z"].iloc[0])

        band_stats = {}
        peak_info = {}

        for filt in ["u", "g", "r", "i", "z", "y"]:
            g = grp[grp["Filter"] == filt]
            if len(g) < 3:
                continue
            t = g["Time (MJD)"].values
            fl = g["Flux_Corr"].values
            er = g["Flux_err"].values
            feats, t_peak, f_peak = extract_band_features(t, fl, er, filt, z)
            row.update(feats)
            if t_peak is not None:
                band_stats[filt] = {
                    "mean": feats.get(f"{filt}_mean", 0.0),
                    "max": feats.get(f"{filt}_max", 0.0),
                    "peak_time": t_peak,
                }
                peak_info[filt] = (t_peak, f_peak)

        # Cross‑band colors + time lag
        bands = ["u", "g", "r", "i", "z", "y"]
        for i in range(len(bands)):
            for j in range(i + 1, len(bands)):
                b1, b2 = bands[i], bands[j]
                if b1 in band_stats and b2 in band_stats:
                    row[f"{b1}_{b2}_mean_diff"] = band_stats[b1]["mean"] - band_stats[b2]["mean"]
                    row[f"{b1}_{b2}_max_diff"] = band_stats[b1]["max"] - band_stats[b2]["max"]
                    row[f"{b1}_{b2}_max_ratio"] = band_stats[b1]["max"] / (band_stats[b2]["max"] + 1e-9)
                    row[f"{b1}_{b2}_lag"] = band_stats[b1]["peak_time"] - band_stats[b2]["peak_time"]

        # Global features
        row["baseline_rest"] = (grp["Time (MJD)"].max() - grp["Time (MJD)"].min()) / (1.0 + max(z, 0.0))
        row["global_snr_max"] = (grp["Flux_Corr"] / (grp["Flux_err"] + 1e-9)).max()
        if "r" in peak_info:
            t_peak_r, _ = peak_info["r"]
            row["frac_after_peak_r"] = (grp["Time (MJD)"] > t_peak_r).mean()
        else:
            row["frac_after_peak_r"] = 0.0
        row["n_obs"] = len(grp)
        row["n_filters_used"] = grp["Filter"].nunique()
        row["frac_neg_flux"] = (grp["Flux_Corr"] < 0).mean()

        features.append(row)

    df_feats = pd.DataFrame(features)
    return df_feats.fillna(-999)

# ========== 5. THỰC HIỆN TRÍCH XUẤT ==========

print("📥 Đang tải lightcurves...")
train_files = glob.glob(os.path.join(BASE_PATH, "**", "train_full_lightcurves.csv"), recursive=True)
test_files = glob.glob(os.path.join(BASE_PATH, "**", "test_full_lightcurves.csv"), recursive=True)

train_log = pd.read_csv(os.path.join(BASE_PATH, "train_log.csv"))
test_log = pd.read_csv(os.path.join(BASE_PATH, "test_log.csv"))

train_lc = pd.concat([pd.read_csv(f) for f in train_files], ignore_index=True).dropna(subset=["Flux"])
test_lc = pd.concat([pd.read_csv(f) for f in test_files], ignore_index=True).dropna(subset=["Flux"])

print("⚙️ Đang tạo đặc trưng tập Train...")
train_feats = process_pipeline(train_lc, train_log)

print("⚙️ Đang tạo đặc trưng tập Test...")
test_feats = process_pipeline(test_lc, test_log)

print("🔗 Đang gộp thông tin log...")
X_full = train_log.gộp(train_feats, on="object_id", how="left").fillna(-999)
X_test_final = test_log.gộp(test_feats, on="object_id", how="left").fillna(-999)

cols = [
    c for c in X_full.columns
    if c not in ["object_id", "target", "split", "SpecType", "English Translation"]
    and pd.api.types.is_numeric_dtype(X_full[c])
]
for c in cols:
    if c not in X_test_final.columns:
        X_test_final[c] = -999

print(f"✅ XONG. Tổng số features: {len(cols)}")
del train_lc, test_lc
gc.collect()


📂 Đang tìm dữ liệu MALLORN...
BASE_PATH: /kaggle/input/mallorn-dataset
📥 Đang tải lightcurves...
⚙️ Đang tạo đặc trưng tập Train...
 -> Trích xuất features...
⚙️ Đang tạo đặc trưng tập Test...
 -> Trích xuất features...
🔗 Đang gộp thông tin log...
✅ XONG. Tổng số features: 360


0

In [3]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import gc

# cấu hình
TOP_K = 140
SCALE_POS_BASE = 1.5
N_SEEDS = 3
LIST_SEEDS = [42, 2024, 777]
THRESH_MIN = 0.30
THRESH_MAX = 0.80
THRESH_STEP = 0.01

# 1. Chuẩn bị dữ liệu
all_features = [
    c for c in X_full.columns
    if c not in ["object_id", "target", "split", "SpecType", "English Translation"]
    and pd.api.types.is_numeric_dtype(X_full[c])
]
X_all = X_full[all_features]
y = X_full["target"].astype(int)
X_test_all = X_test_final[all_features]
print(f"📦 Dữ liệu gốc: {X_all.shape[1]} features")

# Feature selection 1 lần (seed 42)
fs_model = lgb.LGBMClassifier(
    n_estimators=800,
    learning_rate=0.05,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.7,
    objective="binary",
    is_unbalance=True,
    random_state=42,
    n_jobs=-1,
)
fs_model.fit(X_all, y)
importances = fs_model.feature_importances_
indices = np.argsort(importances)[::-1]
top_features = [all_features[i] for i in indices[:TOP_K]]
print(f"✅ Chọn {len(top_features)} features.")
print(" Top 5:", top_features[:5])

X = X_all[top_features]
X_test = X_test_all[top_features]

# 2. Multi‑seed CV
n_pos = (y == 1).sum()
n_neg = (y == 0).sum()
ratio = n_neg / n_pos
scale_weight = ratio * SCALE_POS_BASE
print(f"TDE: {n_pos}, non-TDE: {n_neg}, scale_pos_weight≈{scale_weight:.1f}")

oof_seeds = np.zeros((len(X), N_SEEDS))
test_seeds = np.zeros((len(X_test), N_SEEDS))

for si, seed in enumerate(LIST_SEEDS):
    print(f"\n🚀 Seed {seed}")
    lgb_params = {
        "objective": "binary",
        "metric": "auc",
        "learning_rate": 0.03,
        "num_leaves": 63,
        "max_depth": -1,
        "feature_fraction": 0.55,
        "bagging_fraction": 0.8,
        "bagging_freq": 1,
        "lambda_l1": 1.5,
        "lambda_l2": 4.0,
        "scale_pos_weight": scale_weight,
        "verbosity": -1,
        "seed": seed,
    }

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    oof_pred = np.zeros(len(X))
    test_pred = np.zeros(len(X_test))

    for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y)):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        train_set = lgb.Dataset(X_tr, label=y_tr)
        valid_set = lgb.Dataset(X_va, label=y_va)

        callbacks = [
            lgb.early_stopping(stopping_rounds=200, verbose=False),
        ]

        model = lgb.train(
            lgb_params,
            train_set,
            num_boost_round=4000,
            valid_sets=[valid_set],
            callbacks=callbacks,
        )

        val_pred = model.predict(X_va, num_iteration=model.best_iteration)
        oof_pred[va_idx] = val_pred
        test_pred += model.predict(X_test, num_iteration=model.best_iteration) / kf.n_splits

        tmp_f1 = f1_score(y_va, (val_pred > 0.5).astype(int))
        print(f" Seed {seed} - Fold {fold+1}: best_iter={model.best_iteration}, F1@0.5={tmp_f1:.4f}")

        del model, X_tr, X_va
        gc.collect()

    oof_seeds[:, si] = oof_pred
    test_seeds[:, si] = test_pred

# 3. Ensemble + lưu OOF
oof_final = oof_seeds.mean(axis=1)
test_final = test_seeds.mean(axis=1)

np.save("oof_pred.npy", oof_final)
np.save("y_true.npy", y.values)

# tạm tìm threshold tốt nhất trên OOF (dùng trong báo cáo)
best_f1, best_t = 0.0, 0.5
for t in np.arange(THRESH_MIN, THRESH_MAX, THRESH_STEP):
    preds_bin = (oof_final > t).astype(int)
    score = f1_score(y, preds_bin)
    if score > best_f1:
        best_f1, best_t = score, t

print(f"\n🏆 OOF F1 tốt nhất: {best_f1:.4f} @ t={best_t:.3f}")

# 4. File Nộp Bài với threshold OOF hoặc threshold bạn chọn (vd 0.42)
THRESH_SUB = 0.42  # để đúng báo cáo
sub = pd.DataFrame({
    "object_id": X_test_final["object_id"],
    "target": (test_final > THRESH_SUB).astype(int),
})
sub.to_csv("submission_multiseed_t0.42.csv", index=False)
print(f"✅ Lưu submission_multiseed_t0.42.csv (TDEs: {sub['target'].sum()})")


📦 Dữ liệu gốc: 360 features
[LightGBM] [Info] Number of positive: 148, number of negative: 2895
[LightGBM] [Info] Tự động chọn đa luồng theo cột, the overhead of testing was 0.010808 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Tổng số Bin 89843
[LightGBM] [Info] Số điểm dữ liệu in the train set: 3043, number of used features: 359
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.048636 -> initscore=-2.973528
[LightGBM] [Info] Bắt đầu huấn luyện từ điểm -2.973528
[LightGBM] [Warning] Không còn phân chia nào giúp tăng gain, best gain: -inf
[LightGBM] [Warning] Không còn phân chia nào giúp tăng gain, best gain: -inf
[LightGBM] [Warning] Không còn phân chia nào giúp tăng gain, best gain: -inf
[LightGBM] [Warning] Không còn phân chia nào giúp tăng gain, best gain: -inf
[LightGBM] [Warning] Không còn phân chia nào giúp tăng gain, best gain: -inf
[LightGBM] [Warning] Không còn phân chia nào giúp tăng gain, best gain: -inf
[LightGBM] [Warning] Không còn

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

oof_pred = np.load("oof_pred.npy")
y_true = np.load("y_true.npy")

THRESH_MIN = 0.30
THRESH_MAX = 0.80
THRESH_STEP = 0.01

rows = []
best_f1, best_t = 0.0, 0.5
for t in np.arange(THRESH_MIN, THRESH_MAX, THRESH_STEP):
    preds_bin = (oof_pred > t).astype(int)
    f1 = f1_score(y_true, preds_bin)
    rows.append({"threshold": t, "f1_oof": f1})
    if f1 > best_f1:
        best_f1, best_t = f1, t

df_thresh = pd.DataFrame(rows)
df_thresh.to_csv("debug_oof_threshold_scan.csv", index=False)

print("="*40)
print(f"Best OOF F1 = {best_f1:.4f} @ t = {best_t:.3f}")
print("="*40)
print("Đã lưu debug_oof_threshold_scan.csv để dùng cho bảng + vẽ hình.")


Best OOF F1 = 0.5951 @ t = 0.300
Đã lưu debug_oof_threshold_scan.csv để dùng cho bảng + vẽ hình.


In [6]:
"""
VISUALIZATION SCRIPT FOR MALLORN REPORT (KAGGLE)
Sinh các hình cho báo cáo: 1.1, 3.1, 3.2, 6.1, 6.2 + bảng summary
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score
import warnings
warnings.filterwarnings("ignore")

# ============ SETUP ============
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

COLOR_TDE = '#FF5459'
COLOR_NON_TDE = '#A7A9A9'
COLOR_PRIMARY = '#2180A7'
COLOR_SECONDARY = '#E68161'
COLOR_NEUTRAL = '#5E5240'

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

DPI = 300
OUTPUT_FORMAT = 'png'

DATA_PATH = "/kaggle/input/mallorn-dataset"   # đổi nếu dataset tên khác

# ============ HÌNH 1.1 ============
def plot_himg_1_1():
    """
    Hình 1.1: Class Distribution Bar Chart
    """
    train_log = pd.read_csv(f"{DATA_PATH}/train_log.csv")
    counts = train_log["target"].value_counts().sort_index()
    # 0 = non-TDE, 1 = TDE
    classes = ["non-TDE", "TDE"]
    vals = [counts.get(0, 0), counts.get(1, 0)]
    percentages = [100 * v / sum(vals) for v in vals]
    colors = [COLOR_NON_TDE, COLOR_TDE]

    fig, ax = plt.subplots(figsize=(8, 5), dpi=DPI)
    bars = ax.bar(classes, vals, color=colors, alpha=0.8,
                  edgecolor="black", linewidth=1.5)

    for bar, count, pct in zip(bars, vals, percentages):
        h = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., h,
                f"{count}\n({pct:.2f}%)",
                ha="center", va="bottom",
                fontsize=12, fontweight="bold")

    ax.set_ylabel("Number of Objects", fontsize=12, fontweight="bold")
    ax.set_title("Class Distribution in Training Set",
                 fontsize=14, fontweight="bold", pad=20)
    ax.set_ylim(0, max(vals) * 1.15)
    ax.grid(axis="y", alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"Hinh_1_1_class_distribution.{OUTPUT_FORMAT}",
                dpi=DPI, bbox_inches="tight")
    print("✅ Saved: Hinh_1_1_class_distribution.png")
    plt.show()

# ============ HÌNH 3.1 ============
def plot_himg_3_1():
    """
    Hình 3.1: Target class pie chart
    """
    train_log = pd.read_csv(f"{DATA_PATH}/train_log.csv")
    counts = train_log["target"].value_counts().sort_index()
    tde = counts.get(1, 0)
    non_tde = counts.get(0, 0)
    total = tde + non_tde

    sizes = [tde, non_tde]
    labels = [
        f"TDE\n({100*tde/total:.2f}%)",
        f"non-TDE\n({100*non_tde/total:.2f}%)"
    ]
    colors = [COLOR_TDE, COLOR_NON_TDE]
    explode = (0.15, 0)

    fig, ax = plt.subplots(figsize=(8, 6), dpi=DPI)
    wedges, texts, autotexts = ax.pie(
        sizes,
        labels=labels,
        colors=colors,
        autopct="%1.1f%%",
        explode=explode,
        startangle=90,
        textprops={"fontsize": 11, "weight": "bold"},
        wedgeprops={"edgecolor": "black", "linewidth": 1.5},
    )

    for autotext in autotexts:
        autotext.set_color("white")
        autotext.set_fontsize(12)
        autotext.set_weight("bold")

    ax.set_title("Target Class Distribution",
                 fontsize=14, fontweight="bold", pad=20)

    plt.tight_layout()
    plt.savefig(f"Hinh_3_1_pie_chart.{OUTPUT_FORMAT}",
                dpi=DPI, bbox_inches="tight")
    print("✅ Saved: Hinh_3_1_pie_chart.png")
    plt.show()

# ============ HÌNH 3.2 ============
def plot_himg_3_2():
    """
    Hình 3.2: Histogram of Redshift (Z) and Extinction (EBV)
    """
    train_log = pd.read_csv(f"{DATA_PATH}/train_log.csv")

    fig, axes = plt.subplots(1, 2, figsize=(14, 5), dpi=DPI)

    axes[0].hist(train_log["Z"], bins=25, color=COLOR_PRIMARY,
                 alpha=0.7, edgecolor="black")
    axes[0].set_xlabel("Redshift (Z)", fontsize=12, fontweight="bold")
    axes[0].set_ylabel("Frequency", fontsize=12, fontweight="bold")
    axes[0].set_title("Distribution of Redshift",
                      fontsize=12, fontweight="bold")
    axes[0].grid(axis="y", alpha=0.3)

    axes[1].hist(train_log["EBV"], bins=25, color=COLOR_SECONDARY,
                 alpha=0.7, edgecolor="black")
    axes[1].set_xlabel("Extinction (EBV)", fontsize=12, fontweight="bold")
    axes[1].set_ylabel("Frequency", fontsize=12, fontweight="bold")
    axes[1].set_title("Distribution of Extinction",
                      fontsize=12, fontweight="bold")
    axes[1].grid(axis="y", alpha=0.3)

    fig.suptitle("Distribution of Redshift and Extinction",
                 fontsize=14, fontweight="bold", y=1.02)

    plt.tight_layout()
    plt.savefig(f"Hinh_3_2_redshift_extinction.{OUTPUT_FORMAT}",
                dpi=DPI, bbox_inches="tight")
    print("✅ Saved: Hinh_3_2_redshift_extinction.png")
    plt.show()

# ============ HÌNH 6.1 ============
def plot_himg_6_1():
    """
    Hình 6.1: F1-score vs Threshold (OOF)
    Requires: oof_pred.npy, y_true.npy in /kaggle/working
    """
    oof_pred = np.load("oof_pred.npy")
    y_true = np.load("y_true.npy")

    thresholds = np.arange(0.30, 0.51, 0.01)
    f1_scores = []
    for t in thresholds:
        preds_bin = (oof_pred >= t).astype(int)
        f1_scores.append(f1_score(y_true, preds_bin))

    fig, ax = plt.subplots(figsize=(11, 6), dpi=DPI)
    ax.plot(thresholds, f1_scores, "o-",
            color=COLOR_PRIMARY, linewidth=2.5,
            markersize=6, label="OOF CV")

    max_idx = np.argmax(f1_scores)
    max_t = thresholds[max_idx]
    max_f1 = f1_scores[max_idx]
    ax.scatter([max_t], [max_f1], s=200, color=COLOR_PRIMARY,
               marker="*", edgecolor="black", linewidth=2, zorder=5)
    ax.annotate(f"Max: {max_f1:.4f}\n@ t={max_t:.2f}",
                xy=(max_t, max_f1),
                xytext=(max_t-0.05, max_f1-0.02),
                fontsize=11, fontweight="bold",
                bbox=dict(boxstyle="round,pad=0.5",
                          facecolor=COLOR_PRIMARY, alpha=0.3))

    ax.set_xlabel("Threshold", fontsize=12, fontweight="bold")
    ax.set_ylabel("F1-score", fontsize=12, fontweight="bold")
    ax.set_title("F1-score vs Threshold (OOF)", fontsize=14,
                 fontweight="bold", pad=20)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0.29, 0.51)
    ax.set_ylim(min(f1_scores)*0.95, max(f1_scores)*1.02)

    plt.tight_layout()
    plt.savefig("Hinh_6_1_threshold_optimization.png",
                dpi=DPI, bbox_inches="tight")
    print("✅ Saved: Hinh_6_1_threshold_optimization.png")
    plt.show()

# ============ HÌNH 6.2 ============
def plot_himg_6_2(threshold=0.42):
    """
    Hình 6.2: Confusion Matrix tại threshold cho sẵn (mặc định 0.42)
    Requires: oof_pred.npy, y_true.npy
    """
    oof_pred = np.load("oof_pred.npy")
    y_true = np.load("y_true.npy")
    y_pred = (oof_pred >= threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred)

    fig, ax = plt.subplots(figsize=(8, 7), dpi=DPI)
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues",
        xticklabels=["non-TDE", "TDE"],
        yticklabels=["non-TDE", "TDE"],
        cbar_kws={"label": "Count"},
        ax=ax,
        linewidths=2,
        linecolor="black",
    )

    ax.set_xlabel("Predicted Label", fontsize=12, fontweight="bold")
    ax.set_ylabel("True Label", fontsize=12, fontweight="bold")
    ax.set_title(f"Confusion Matrix (Threshold = {threshold})",
                 fontsize=14, fontweight="bold", pad=20)

    tn, fp, fn, tp = cm.ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    metrics_text = (
        f"Precision: {precision:.4f}\n"
        f"Recall: {recall:.4f}\n"
        f"F1-score: {f1:.4f}"
    )
    ax.text(0.98, 0.02, metrics_text, transform=ax.transAxes,
            fontsize=11, va="bottom", ha="right",
            bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.8))

    plt.tight_layout()
    plt.savefig("Hinh_6_2_confusion_matrix.png",
                dpi=DPI, bbox_inches="tight")
    print("✅ Saved: Hinh_6_2_confusion_matrix.png")
    plt.show()

# ============ BẢNG SUMMARY ============
def create_summary_table():
    data = {
        "Stage": ["Baseline", "Advanced FE", "Final (Multi-seed)"],
        "F1 OOF": ["~0.55", "0.6033", "0.6073"],
        "Public F1": ["0.56", "0.59", "0.6228"],
        "Improvement": ["-", "+0.03", "+0.0328"],
        "Key Changes": [
            "Basic stats only",
            "TDE fit + Cross-band + BIẾN ĐỔI FOURIER",
            "Multi-seed ensemble + Rest-frame",
        ],
    }
    df_summary = pd.DataFrame(data)

    fig, ax = plt.subplots(figsize=(14, 3), dpi=DPI)
    ax.axis("tight")
    ax.axis("off")

    table = ax.table(
        cellText=df_summary.values,
        colLabels=df_summary.columns,
        cellLoc="center",
        loc="center",
        colWidths=[0.15, 0.15, 0.15, 0.15, 0.4],
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2.5)

    for i in range(len(df_summary.columns)):
        table[(0, i)].set_facecolor(COLOR_PRIMARY)
        table[(0, i)].set_text_props(weight="bold", color="white")

    plt.title("Model Performance Improvement Summary",
              fontsize=14, fontweight="bold", pad=20)
    plt.savefig(f"Table_summary_statistics.{OUTPUT_FORMAT}",
                dpi=DPI, bbox_inches="tight")
    print("✅ Saved: Table_summary_statistics.png")
    plt.show()

# ============ MAIN ============
def main():
    print("=" * 60)
    print("🎨 MALLORN VISUALIZATION GENERATOR (KAGGLE)")
    print("=" * 60)

    plot_himg_1_1()
    plot_himg_3_1()
    plot_himg_3_2()
    create_summary_table()
    plot_himg_6_1()
    plot_himg_6_2(threshold=0.42)

    print("\n✅ All figures generated! Check /kaggle/working")

if __name__ == "__main__":
    main()


🎨 MALLORN VISUALIZATION GENERATOR (KAGGLE)
✅ Saved: Hinh_1_1_class_distribution.png


✅ Saved: Hinh_3_1_pie_chart.png


✅ Saved: Hinh_3_2_redshift_extinction.png


✅ Saved: Table_summary_statistics.png


✅ Saved: Hinh_6_1_threshold_optimization.png


✅ Saved: Hinh_6_2_confusion_matrix.png



✅ All figures generated! Check /kaggle/working


In [8]:
import os, zipfile

zip_name = "mallorn_figures.zip"

with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as z:
    for fname in os.listdir("/kaggle/working"):
        if fname.lower().endswith(".png"):   # nếu muốn cả csv thì thêm ", '.csv'"
            z.write(os.path.join("/kaggle/working", fname), fname)

print("Đã tạo:", zip_name)


Đã tạo: mallorn_figures.zip
